In [23]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import warnings
warnings.filterwarnings('ignore')

# Data Cleaning


In [24]:
def correctcol(data):
    new_header = data.iloc[0] 
    data = data[1:] 
    data.columns = new_header
    data = data.loc[:, data.columns.notnull()]
    return data
def filtercustid(data):
    final = data.loc[data['customer_id'] <= 3500]
    return final
def cleantrans(data):
    data = correctcol(data)
    data = filtercustid(data)
    data = data.loc[data['order_status'] != 'Cancelled']
    data['profit'] = data['list_price'] - data['standard_cost']
    data['online_order'].replace(np.nan, 'unknown',inplace=True)
    data['brand'].replace('', np.nan, inplace=True)
    data.dropna(subset=['brand'], inplace=True)
    return data
def cleancustadd(data):
    data = correctcol(data)
    data["state"].replace({"New South Wales": "NSW", "Victoria": "VIC"}, inplace=True)
    return data
def cleancustdemo(data):
    data = correctcol(data)
    data = filtercustid(data)
    a = data.loc[data['deceased_indicator'] != 'Y']
    a.replace({"Male": "M", "Female": "F", "Femal":"F"}, inplace=True)
    a['DOB'] = pd.to_datetime(a['DOB'], errors='coerce')
    now = dt.now()
    a['age']= (now.year - a['DOB'].dt.year) - ((now.month - a['DOB'].dt.month) < 0)
    del a['default']
    a['job_title'].replace(np.nan, 'unknown',inplace=True)
    a['job_industry_category'].replace(np.nan, 'unknown',inplace=True)
    a['tenure'].replace('', np.nan, inplace=True)
    a.dropna(subset=['tenure'], inplace=True)
    del a['DOB']
    return a

In [25]:
trans = pd.read_excel('KPMG.xlsx','Transactions')
custadd = pd.read_excel('KPMG.xlsx', 'CustomerAddress')
custdemo= pd.read_excel('KPMG.xlsx', 'CustomerDemographic')

In [26]:
custadd = cleancustadd(custadd)
trans =  cleantrans(trans)
custdemo = cleancustdemo(custdemo)


In [27]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

#make a lists, indicating which features
# will be imputed with each method

features_numeric = ['age']

# then instantiate the imputers, within a pipeline
# we create one imputer for numerical and one imputer
# for categorical

# this imputer imputes with the mean
imputer_numeric = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
])



# then put the features list and the transformers together
# using the column transformer

preprocessor = ColumnTransformer(transformers=[('imputer_numeric',
                                                imputer_numeric,
                                                features_numeric)])

# now fit the preprocessor
preprocessor.fit(custdemo[['age']])

# and now can impute the data
# remember it returs a numpy array

custdemo['age'] = preprocessor.transform(custdemo[['age']]).ravel()
# X_test = preprocessor.transform(X_test)

#  Data preparation

In [28]:
 final=[custdemo,custadd,trans]

In [32]:
# merging three dataframes 
from functools import reduce
merged = reduce(lambda left,right: pd.merge(left,right,on='customer_id'), final)

In [33]:
merged

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,...,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date,profit
0,1,Laraine,Medendorp,F,93,Executive Secretary,Health,Mass Customer,N,Yes,...,False,Approved,OHM Cycles,Standard,medium,medium,235.63,125.07,38482,110.56
1,1,Laraine,Medendorp,F,93,Executive Secretary,Health,Mass Customer,N,Yes,...,True,Approved,Solex,Standard,medium,medium,1577.53,826.51,39526,751.02
2,1,Laraine,Medendorp,F,93,Executive Secretary,Health,Mass Customer,N,Yes,...,True,Approved,Trek Bicycles,Road,low,small,1720.7,1531.42,37823,189.28
3,1,Laraine,Medendorp,F,93,Executive Secretary,Health,Mass Customer,N,Yes,...,False,Approved,Norco Bicycles,Standard,medium,medium,360.4,270.3,37873,90.1
4,1,Laraine,Medendorp,F,93,Executive Secretary,Health,Mass Customer,N,Yes,...,False,Approved,Solex,Standard,medium,medium,71.49,53.62,38573,17.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19147,3500,Josy,Fleeman,F,71,unknown,Entertainment,Affluent Customer,N,No,...,True,Approved,Giant Bicycles,Road,medium,medium,792.9,594.68,33879,198.22
19148,3500,Josy,Fleeman,F,71,unknown,Entertainment,Affluent Customer,N,No,...,False,Approved,WeareA2B,Standard,medium,medium,1228.07,400.91,36668,827.16
19149,3500,Josy,Fleeman,F,71,unknown,Entertainment,Affluent Customer,N,No,...,False,Approved,Giant Bicycles,Standard,medium,small,1311.44,1167.18,33888,144.26
19150,3500,Josy,Fleeman,F,71,unknown,Entertainment,Affluent Customer,N,No,...,False,Approved,WeareA2B,Standard,medium,medium,60.34,45.26,34165,15.08
